In [1]:
from pyspecter.SPECTER import SPECTER
from pyspecter.Observables import Observable
# from pyspecter.SpecialObservables import SpecialObservables

# Utils
from pyspecter.utils.data_utils import load_cmsopendata, load_triangles

# Jax
from jax import random
import jax.numpy as jnp


2023-10-27 11:51:29.052708: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
No GPU/TPU found, falling back to CPU. (Set TF_CPP_MIN_LOG_LEVEL=0 and rerun for more info.)
/home/rikab/miniconda3/envs/SHAPER/lib/python3.8/site-packages/ot/backend.py:2998: UserWarning: To use TensorflowBackend, you need to activate the tensorflow numpy API. You can activate it by running: 
from tensorflow.python.ops.numpy_ops import np_config
np_config.enable_numpy_behavior()
  register_backend(TensorflowBackend())


In [2]:
dataset_open, sim_weights, k = load_cmsopendata("~/.energyflow/", "sim", 475, 525, 1.9, 2, pad = 75, n = 1000)
triangle_events, triangle_indices = load_triangles(10, 10, R = 1.0, return_indices=True)

Max # of particles: 124


In [3]:
specter = SPECTER(compile=True)

Compiling SPECTER model...
Generating test events for tracing ...
Test events generated! Time taken:  1.4871983528137207  seconds.
Compiling observables...
Observables compiled! Time taken:  1.487248420715332  seconds.
Compiling spectral representation functions ...
Compilation complete! Time taken:  27.01432156562805  seconds.


In [4]:
# Dataset
R = 1.0
beta = 1.0
triangle_events, triangle_indices = load_triangles(num_angles=180, num_energies= 100, R = 1.0, return_indices=True)

# Define the ring sampler
def ring_sampler(Radius = 1, N = 150, seed = 0):

        # Random seed
        key = random.PRNGKey(seed)
        key1, key2 = random.split(key)

        # For a spiral -- use key 1 = key 2!
        # Uncomment the following to get a disk instead
        phi = 2 * jnp.pi * random.uniform(key1, shape = (N,))
        r = 1 #jnp.sqrt( random.uniform(key2, shape = (N,)))

        x = Radius * r * jnp.cos(phi)
        y = Radius * r * jnp.sin(phi)
        e = jnp.ones_like(r) / N
        points = jnp.stack((e, x, y), axis = -1)

        return points

# Define two observables: Generic sRinginess estimate and the closed-form expression
spRinginess1 = Observable({"Radius" : 0.5}, ring_sampler, name = "sRinginess1")
spRinginess2 = SpecialObservables["spRinginess"]

# Prepare SPECTER and calculate observables using default setup
specter = SPECTER(observables=[spRinginess1, spRinginess2])
out1 = specter.compute_spRinginess1(triangle_events, beta = 1.0, R = 1.0, timing = True)
out2 = specter.compute_spRinginess2(triangle_events, beta = 1.0, R = 1.0, timing = True)

# Plot
plt.scatter(out1["sEMD"], out1["Radius"], color = "red")
plt.scatter(out2["sEMD"], out2["Radius"], color = "blue")



TypeError: __init__() got an unexpected keyword argument 'name'

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

ss = specter.compute_spectral_representation(triangles)
print(specter.spectralEMD(ss, ss, type1="spectral", type2="spectral"))
print(specter.spectralEMD(triangle_events, triangle_events))



aaa (3, 3)
[ 0.0000000e+00  0.0000000e+00 -5.9604645e-08  1.1920929e-07
 -5.9604645e-08 -5.9604645e-08  0.0000000e+00 -5.9604645e-08
  1.1920929e-07  0.0000000e+00  0.0000000e+00  0.0000000e+00
 -1.1920929e-07  1.1920929e-07 -1.1920929e-07 -5.9604645e-08
  0.0000000e+00 -5.9604645e-08  1.1920929e-07  0.0000000e+00
  0.0000000e+00  0.0000000e+00  0.0000000e+00  1.1920929e-07
 -1.1920929e-07 -1.1920929e-07  0.0000000e+00 -1.1920929e-07
  1.1920929e-07  0.0000000e+00  0.0000000e+00  0.0000000e+00
  0.0000000e+00  2.3841858e-07 -1.1920929e-07  1.1920929e-07
 -1.1920929e-07  0.0000000e+00  1.1920929e-07  0.0000000e+00
  0.0000000e+00 -2.3841858e-07  0.0000000e+00  2.3841858e-07
 -1.1920929e-07  2.3841858e-07  0.0000000e+00  0.0000000e+00
  3.5762787e-07  0.0000000e+00  0.0000000e+00  0.0000000e+00
  0.0000000e+00  2.3841858e-07 -2.3841858e-07  4.7683716e-07
 -2.3841858e-07  5.9604645e-08  7.1525574e-07  0.0000000e+00
  0.0000000e+00 -4.7683716e-07 -4.7683716e-07  1.4305115e-06
 -4.7683716e-

In [ ]:
# TODO: Fix JIT compiler

ds2_events1_spectral2 = SPECTER.ds2_events1_spectral2
